# <mark>Novo Processo de Pipeline - 1_Extruturacao_Pipeline_docs.ipynb</mark>

Atual notebook com as funçoes para processamento de PDF Pesquisavel e Raster e a criaçao dos Dataframes de forma dependente e unica

## P1 Extração

### P1 - Modules e config

In [121]:
import os
import sys
import shutil
import platform
from io import StringIO
from pathlib import Path
from urllib import response

from outlook_msg import Message
import extract_msg
import zipfile
from pyunpack import Archive
import py7zr

import re
from unidecode import unidecode
from unicodedata import normalize

import csv
import json
import pandas as pd


import uuid
import hashlib

import locale
import time, copy
from pytz import timezone
from datetime import datetime, timezone, timedelta

import cv2
import fitz  # Módulo PyMuPDF
from PIL import Image
from PIL import ImageFont
from PIL import Image, ImageDraw
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer, LTChar
from pdf2image import convert_from_path
import matplotlib.pyplot as plt

import pytesseract

# Modulos da solucao
import modules.extrai_pdf_pesquisavel as Extc
import modules.cronometro as cron

In [122]:
#### Config - E-mail

# 1. Caminho do arquivo uma mensagem especifica
msg_dir_path = 'pipeline_extracao_documentos/1_emails_documentos_recebidos/11_emails'

# 2. Path para arquivos atachados compactados
msg_attachment_zip = 'pipeline_extracao_documentos/1_emails_documentos_recebidos/12_attachments'

# 3. Path para arquivos recebidos manualmente
arquivos_recebidos_path = 'pipeline_extracao_documentos/1_emails_documentos_recebidos/12_attachments'

# 4. Path para documentos para extracao
documentos_extracao_path = "pipeline_extracao_documentos/2_documentos_para_extracao/21_aguardando_processamento"




# 6. Path para gestao de imagens resized
image_resized_path = "pipeline_extracao_documentos/6_geral_administacao/temp_docs/images/processadas"


# 1. Path para DFs e CSVs exportados
export_path = "pipeline_extracao_documentos/6_geral_administacao/exports"

# 2. Path para lixeira
root_garbage_path = "pipeline_extracao_documentos/0_lixeira"


### funcoes para teste

In [123]:
# 1. XXX Funcao flush_documents
def flush_documents(i, folder_name, file, file4garbage_path, new_path_file4garbage):
    
    # IMPORTANTE: Coloque aqui sua lógica para cada arquivo
    time_now = cron.timenow_pt_BR()
    status_process = "MOVIDO"
    
    return {
        'id': i,
        'data_eliminacao':time_now, 
        'diretorio': folder_name,
        'arquivo': file,
        'path_origem': file4garbage_path,
        'path_destino': new_path_file4garbage,
        'status': status_process,

    }


param_garbage_path = root_garbage_path
# 2. FunÇao para mover documentos de um diretorio para a lixeira
def garbage_pipe(docs4garbage_path, param_garbage_path=root_garbage_path):
    rows_list = []   
    i = 1
    for root, dirs, files in os.walk(docs4garbage_path):
        folder_name = os.path.basename(root)
        for file in files:
            # 1. Defino o path do arquivo que sera mandado para lixeira
            file4garbage_path = os.path.join(root, file)
            # 2. Defino dsubdiretorio para envio de documentos
            garbage_subdir_path = os.path.join(param_garbage_path, folder_name)
            
            # 3. Crio o subdiretorio em garbage
            if not os.path.exists(garbage_subdir_path):
                os.makedirs(garbage_subdir_path)
            
            # 4. Defino o path final do arquivo para a lixeira
            new_path_file4garbage = os.path.join(garbage_subdir_path, file)
            # 5. Processo a transferencia
            try:
                shutil.move(file4garbage_path, new_path_file4garbage)
                print(f'p{i:>3} | move: {file} | for: {new_path_file4garbage}')
                new_row = flush_documents(i, folder_name, file, file4garbage_path, new_path_file4garbage)
                rows_list.append(new_row)
            except Exception as e:
                print(f"Erro ao mover documento {file}: {e}")
            folder_to_remove = os.path.join(docs4garbage_path, folder_name)
            i += 1 
               
    os.rmdir(folder_to_remove)
    
    df_garbage = pd.DataFrame(rows_list)
    

            
            #print("1. path da lixeira:", param_garbage_path)
    print(i-1, "Documentos movidos")
    
    
    # 2. Tratando nome de carga do df_extracao_files_Batch
    df_garbage_file = "df_garbage_"
    df_filename_wr = df_garbage_file + str(folder_name) + ".xlsx"
    path_to_export_df = os.path.join(export_path, df_filename_wr)
    # Salvando o DF para excel
    df_garbage.to_excel(path_to_export_df, index=False)
    
    return df_garbage

# 3. Buscar o primeiro documento do diretorio EXTERNO para processar
def busca_doc_test(src_dir_path, tipo_file, tgt_dir_path):
 
    extensao = "." + str(tipo_file)
    # Verifique se o diretório de destino está vazio
    if os.listdir(tgt_dir_path):
        raise Exception("O diretório de destino não está vazio!")

    for roots, directories, documents in os.walk(src_dir_path):
        # Filtre os documentos para incluir apenas aqueles com extensão .pdf
        doc_files = [doc for doc in documents if doc.lower().endswith(extensao)]
        
        if doc_files: # Verifique se há algum arquivo PDF no diretório
            first_doc_file = doc_files[0] # Obtenha o primeiro arquivo PDF
            source_path = os.path.join(roots, first_doc_file) # Construa o caminho completo para o primeiro arquivo PDF
            print(f"Encontrei o primeiro arquivo: {source_path}")
            
            # Mova o arquivo para o diretório de destino
            shutil.move(source_path, tgt_dir_path)
            
            break # Saia do loop após encontrar o primeiro arquivo PDF


def move_pdf_processed_ok(document_path):
    
    source_path = document_path
    destination_path = os.path.join(f'{nf_processada_path}/{str(doc2convert)}')
    shutil.move(source_path, destination_path)

# # 3. parametros e paths para processamento
# param_garbage_path = root_garbage_path
# docs4garbage_path = "pipeline_extracao_documentos/2_documentos_para_extracao/21_aguardando_processamento"
# df_garbage = garbage_pipe(docs4garbage_path, param_garbage_path=root_garbage_path)
# df_garbage  

In [ ]:
# Trazendo o primeiro e-mail para teste
sorce_mail = "pipeline_extracao_documentos/0_arquivos_teste_pipeline/emails"
target_dir_path = "pipeline_extracao_documentos/1_emails_documentos_recebidos/11_emails"

tipo_file = "msg"
busca_doc_test(sorce_mail, tipo_file, target_dir_path)

### P1 - Funcoes para e-mail e extracao

<mark> <b>1.</b> Abrindo o pipeline de documentos do email </mark>

In [4]:
# Geraçao do hash do arquivo
def generate_file_hash(file_path):
    # Abre o arquivo em modo de leitura de bytes
    with open(file_path, "rb") as f:
        # Lê o conteúdo do arquivo
        file_data = f.read()
        # Utiliza o algoritmo SHA-256 para gerar o hash
        file_hash = hashlib.sha256(file_data).hexdigest()
    return file_hash

# Geraçao do Unique_id do arquivo
def generate_unique_id():
    return str(uuid.uuid4())


# 1. XXX Busca proximo Batch
def busca_proximo_batch():
    # Abre o arquivo Excel e lê a coluna 'batch'
    df = pd.read_excel("pipeline_extracao_documentos/6_geral_administacao/exports/df_documento_recebido.xlsx", usecols=["batch"])
    # Pega o último valor da coluna 'batch'
    last_value = df.iloc[-1, 0]
    
    # Extraí o número do último batch e adiciona 1 para o próximo
    last_number = int(last_value.split("_")[1])
    next_number = last_number + 1
    
    # Forma o nome do próximo batch
    next_batch = f"Batch_{next_number}"
    
    return next_batch    

# 2. XXX Criacao do pripeline para processar email
def process_email(msg, origem_docs, first_doc_file):
    
    batch_name = busca_proximo_batch()
    origem = origem_docs
    msg_raw_sender = msg.sender
    parts = msg_raw_sender.rsplit('<', 1)
    msg_email_address = parts[1].strip('<>')
    msg_sender = parts[0].strip(' ')
    msg_subject = msg.subject
    msg_body = msg.body
    # String original
    original_date_str = msg.date
    date_email = cron.convert_email_date(original_date_str)
    
    #status_pro = '...'
    return {
        'dt_hora': date_email,
        'origem': origem,
        'de': msg_sender,
        'assunto': msg_subject,
        'batch': batch_name,
        'email': msg_email_address,
        'msg_file': first_doc_file,
    }, batch_name

# 3. FunÇao para buscar emails e attachments
def email_pipe():
    
    locale.setlocale(locale.LC_TIME, "pt_BR.utf8")
    rows_list = []   
    for root, dirs, files in os.walk(msg_dir_path):
        folder_name = os.path.basename(root)
        doc_files = [doc for doc in files if doc.lower().endswith(".msg")]
        if doc_files: # Verifique se há algum arquivo .msg no diretório
            first_doc_file = doc_files[0] # Obtenha o primeiro arquivo .msg
            email_path = os.path.join(root, first_doc_file) # Construa o caminho completo para o primeiro arquivo .msg
            try:
                origem_docs = 'email'            
                msg = extract_msg.Message(email_path)
                new_row, nome_batch = process_email(msg, origem_docs, first_doc_file)
                
                hash_value = generate_file_hash(email_path)
                
                rows_list.append(new_row)
            except Exception as e:
                print(f"Erro ao ler email: {e}") 
    
    
    df_mail = pd.DataFrame(rows_list)
    # Caminho da pasta onde você quer salvar os anexos
    pasta_destino = msg_attachment_zip

    # Verifica se a pasta existe; se não, cria ela
    if not os.path.exists(pasta_destino):
        os.makedirs(pasta_destino)

    with open(email_path) as msg_file:
        msg = Message(msg_file)
    
    total_attch = len(msg.attachments)
    
    i = 0
    # Loop para salvar cada anexo
    for i in range(total_attch):
        attachment = msg.attachments[i]
        caminho_completo_anexo = os.path.join(pasta_destino, attachment.filename)

        #print(caminho_completo_anexo)
        with attachment.open() as attachment_fp, open(caminho_completo_anexo, 'wb') as output_fp:
            output_fp.write(attachment_fp.read())

    return df_mail, nome_batch

# 4. XXX converte nome do arquivo
def conv_filename(title):
    
    # Divide o título em nome e extensão
    name, extension = title.rsplit('.', 1) if '.' in title else (title, "")

    # Remove acentos e caracteres especiais do nome
    name = normalize('NFKD', name).encode('ASCII', 'ignore').decode('ASCII')
    
    # Substiti espaços e hífens por sublinhados
    filename = name.replace(' ', '_').replace('-', '_')

    # Remove quaisquer outros caracteres não alfanuméricos, exceto sublinhados
    filename = re.sub(r'[^\w_]', '', filename)

    # Converte para minúsculas
    filename = filename.lower()

    # Adiciona a extensão de volta, se houver
    if extension:
        filename += '.' + extension.lower()

    return filename

# 5. XXX Extrai documentos compatados
def extract_documents(src_compressed_file_path, tgt_directory_path, batch):

    # Diretório onde você quer salvar os arquivos extraídos
    output_dir = os.path.join(tgt_directory_path, batch)

    folder_file_dict = {}

    for root, dirs, files in os.walk(src_compressed_file_path):
        #print(f'{root}  | {dirs} | {document} | {files}\n')
        
        for file in files:
            file_path = os.path.join(root, file)

            if file.lower().endswith('.zip'):
                zip_file = file
                zip_file_path = os.path.join(root, file)
                # Obtém o nome base do arquivo ZIP para usar como subdiretório
                zip_base_name = os.path.splitext(os.path.basename(zip_file_path))[0]
                zip_basename = conv_filename(zip_base_name)
                
                # Cria o subdiretório com base no nome do arquivo ZIP
                root_output_dir = os.path.join(output_dir, zip_basename)
                
                if not os.path.exists(root_output_dir):
                    os.makedirs(root_output_dir) # estou criando o diretorio caso nao exista

                # Abre o arquivo ZIP
                with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
                    for member in zip_ref.namelist():
                        # Separa o nome da pasta e o nome do arquivo usando barra invertida como delimitador
                        parts = member.rsplit('\\', 1)
                        folder_name = parts[0] if len(parts) > 1 else ''
                        #folder_name = conv_filename(folder_temp)
                        filename = parts[-1]
                        if filename:  # ignora diretórios
                            # Adiciona ao dicionário
                            #filename = conv_filename(filename)
                            # Cria um subdiretório se ele não existir
                            sub_dir = os.path.join(root_output_dir, folder_name)
                            if not os.path.exists(sub_dir):
                                os.makedirs(sub_dir)

                            # Salva o arquivo no subdiretório especificado
                            source = zip_ref.open(member)
                            target_path = os.path.join(sub_dir, filename)
                            
                            with open(target_path, "wb") as target:
                                target.write(source.read())
                                dir_path = os.path.dirname(filename)
                                
            elif file.lower().endswith('.rar'):
                rar_file = file
                rar_file = conv_filename_no_ext(rar_file)
                rar_file_path = os.path.join(root, file) 
                root_output_dir = os.path.join(output_dir, rar_file)
                if not os.path.exists(root_output_dir):
                    os.makedirs(root_output_dir)
                Archive(rar_file_path).extractall(root_output_dir)  
                
            elif file.lower().endswith('.7z'):
                sevenz_file = file
                sevenz_file = conv_filename_no_ext(sevenz_file)
                sevenz_file_path = os.path.join(root, file)
                root_output_dir = os.path.join(output_dir, sevenz_file)                      
                                
                with py7zr.SevenZipFile(sevenz_file_path, mode='r') as z:
                    z.extractall(root_output_dir)
                    
            elif file.lower().endswith('.pdf'):
                file_path = os.path.join(root, file)
                new_path_name = os.path.join(output_dir, file)
                if not os.path.exists(output_dir):
                                os.makedirs(output_dir)
                shutil.move(file_path, new_path_name)

In [ ]:
# Busca proximo Batch caso nao esteja rodando email
batch_name = busca_proximo_batch()
batch_name

In [5]:
# 1. Processa e-mail e guarda seus anexos
df_mail, batch_name = email_pipe()

In [6]:
df_mail

,dt_hora,origem,de,assunto,batch,email,msg_file
0,10/08/2023 23:20:56,email,Verlânio Gallindo,Fwd: Notas Magé 2,Batch_17,verlanio@gmail.com,Fwd Notas Magé 2.msg


In [7]:
# 2. Efetua a extraçao de documento
extract_documents(msg_attachment_zip, documentos_extracao_path, batch_name) 

In [48]:
# 5. XXX Estabelece o pipeline de documentos
def abertura_pipeline(index, batch_name, file, documentos_extracao_path, folder_name, file_path):
    action_itens = {}
    
    status_pro = "para_analise"
    time_now = cron.timenow_pt_BR()
    
    
    return {
        'index': index,
        'data_processamento': time_now,
        'Batch': batch_name,
        'diretorio_origem': folder_name,
        'nome_arquivo': file,
        'diretorio_origem': file_path,
        'status': status_pro,
    }


# 3. funcao para ajustar documentos no pipeline antes de serem processados
def pipeline_preprocessamento():
    rows_list = []
    i = 0
    for root, dirs, files in os.walk(documentos_extracao_path):
        folder_name = os.path.basename(root)
        for file in files:
            file_path = os.path.join(root, file)
            try:
                i += 1
                new_row = abertura_pipeline(i, batch_name, file, documentos_extracao_path, folder_name, file_path)
                rows_list.append(new_row)
            except Exception as e:
                print(f"Erro ao criar nova row: {e}") 
            
    
    df_documento_recebido = pd.DataFrame(rows_list)
    
    return df_documento_recebido

In [79]:
df_documento_recebido = pipeline_preprocessamento()

In [80]:
df_documento_recebido

,index,data_processamento,Batch,diretorio_origem,nome_arquivo,status
0,1,06/09/2023 18:45:57,Batch_17,pipeline_extracao_documentos/2_documentos_para...,Nota Fiscal Eletrônica 1717 - BALL - BVI.PDF,para_analise
1,2,06/09/2023 18:45:57,Batch_17,pipeline_extracao_documentos/2_documentos_para...,NF 688 - ConsoR Zadar- Engetecnica (ENZA).pdf,para_analise
2,3,06/09/2023 18:45:57,Batch_17,pipeline_extracao_documentos/2_documentos_para...,1127 fogo em 20-07-23.pdf,para_analise
3,4,06/09/2023 18:45:57,Batch_17,pipeline_extracao_documentos/2_documentos_para...,NF 689- PMMacae- 8ª Med CO-22-22 TB Contrato 0...,para_analise
4,5,06/09/2023 18:45:57,Batch_17,pipeline_extracao_documentos/2_documentos_para...,Nota Fiscal_229_Agosto-Schneider.pdf,para_analise
5,6,06/09/2023 18:45:57,Batch_17,pipeline_extracao_documentos/2_documentos_para...,38 EDSON JOSE OLIVEIRA DOS SANTOS.pdf,para_analise
6,7,06/09/2023 18:45:57,Batch_17,pipeline_extracao_documentos/2_documentos_para...,12 - Ótica do Cizinho.pdf,para_analise
7,8,06/09/2023 18:45:57,Batch_17,pipeline_extracao_documentos/2_documentos_para...,FLEXPRIN.pdf,para_analise
8,9,06/09/2023 18:45:57,Batch_17,pipeline_extracao_documentos/2_documentos_para...,NFS-e 35.pdf,para_analise
9,10,06/09/2023 18:45:57,Batch_17,pipeline_extracao_documentos/2_documentos_para...,NFS-e 37.pdf,para_analise


In [81]:
# 2. XXX Analisa nro de paginas
def analisa_nro_pages(file_path):
    
    pdf_document = fitz.open(file_path)
    pages = pdf_document.pages() # generator object

    page_nro = []
    for page in pages:
        page_nro.append(page)
        
    nro_paginas = len(page_nro)    
    if nro_paginas > 1:
        doc_1_page = False
        return doc_1_page, nro_paginas    
    else:
        doc_1_page = True
        return doc_1_page, nro_paginas  
    pdf_document.close() 
    
    
# 3. XXX Split de paginas
def split_pdf_pages(file2split_path):
    
    try:
        pdf = fitz.open(file2split_path)
        # Número total de páginas no PDF
        total_pages = len(pdf)
    except Exception as e:
        print(f"Nao congui abrir o PDF: {e}")    

    # Nome base para os arquivos de saída
    base_name = file2split_path.split('.')[0]  # Remove a extensão do arquivo
    
    file_to_delete = file2split_path

    # Loop para criar um novo PDF para cada página
    for page_num in range(total_pages):
        # Cria um novo objeto PDF
        new_pdf = fitz.open()
        # Adiciona a página atual ao novo PDF
        new_pdf.insert_pdf(pdf, from_page=page_num, to_page=page_num)
        # Nome do novo arquivo PDF
        new_pdf_name = f"{base_name}_page_{page_num + 1}.pdf"
        # Salva o novo PDF
        new_pdf.save(new_pdf_name)
        # Fecha o novo PDF
        new_pdf.close()
    return new_pdf_name, file_to_delete
    # Fecha o PDF original
    pdf.close()   
    
    
# 4. XXX Efetua a conversao e o resize pagina - NF (convem pensar noutro modelo para listagem) 
def convertResizeAnalise_1page(doc2convert, document_path, image_resized_path):
    
    # 1. construo um novo nome para o documento imagem
    image_resized_name = os.path.join(f'{image_resized_path}/{str(doc2convert)}.jpg')
    
    # 2. Conversao para imagem
    pages = convert_from_path(document_path, 500, poppler_path=poppler_path)
    
    # 3. Para cada pagina faco o resize (apesar de ser somente uma)
    resized_pages = []
    for page in pages:
        resized_page = page.resize((2067, 2923))
        resized_pages.append(resized_page)
        resized_pages[0].save(image_resized_name, 'JPEG')
        
    return resized_pages[0], image_resized_name 


# 5. XXX Pesquisa prefeitura no documento (dando as coordenadas) e efetuando o OCR
def pequisaTextoDoc(image_name):

    # 1. Definindo as coordenadas do frame
    x0 = 0
    y0 = 0
    x1= 2066
    y1 = 2922

    # 2. Definir frame_image
    frame_image = image_name.crop((x0, y0, x1, y1))

    # 3. Extraia texto usando OCR com configuração de idioma padrão para este frame
    extracted_text_frame = pytesseract.image_to_string(frame_image, lang='por', config=tessdata_dir_config).strip()
    
    return extracted_text_frame 

# 6.1. XXX Ajusta texto
def texto_extraido(texto):
    #0. Tratamento da string
    text_splited = texto.split('\n')
    text_splited = [s.replace(":", "") for s in text_splited]
    text_splited = [x for x in text_splited if x.strip()]
    text_splited = [s.replace(";", "").strip() for s in text_splited] #depende da situaçao
    return text_splited

# 6.2. XXX Ajusta texto sem quebrar o ":"
def texto_extraido_nf(texto):
    #0. Tratamento da string
    text_splited = texto.split('\n')
    text_splited = [x for x in text_splited if x.strip()]
    text_splited = [s.replace(";", "").strip() for s in text_splited] #depende da situaçao
    return text_splited

In [82]:
def criar_df_analise(df_documento_recebido):
    linhas_df_analise = []
    pre_processo = ['ajustar_nome', 'split_paginas', 'ajustar_imagem', 'buscar_nome_prefeitura', 'enviar_canceladas', 'enviar_listagens']
    
    for idx, row in df_documento_recebido.iterrows():
        file_name = row['nome_arquivo']
        file_path = row['diretorio_origem']
        #file_path = f"{row['diretorio_origem']}/{file_name}"  # Ajustar como necessário

        # Chama a função analisa_nro_pages
        doc_1_page, nro_paginas = analisa_nro_pages(file_path)

        # Chama a função conv_filename
        new_name = conv_filename(file_name)

        acoes_necessarias = {}

        # Se o documento tem mais de uma página
        if not doc_1_page:
            acoes_necessarias['split_paginas'] = nro_paginas

        # Se o nome do arquivo precisa ser ajustado
        if new_name != file_name:
            acoes_necessarias['ajustar_nome'] = new_name
            
        if "cancelada" in file_name.lower():
            acoes_necessarias['enviar_canceladas'] = True
            
        if "listagem" in file_name.lower():
            acoes_necessarias['enviar_listagens'] = True
                

        for action in pre_processo:
            if action in acoes_necessarias:
                info = acoes_necessarias[action]
                seq = pre_processo.index(action) + 1  # Pega a sequência da ação baseada na lista pre_processo
                
                nova_linha = {
                    'index': row['index'],
                    'data_processamento': row['data_processamento'],
                    'Batch': row['Batch'],
                    'nome_arquivo': row['nome_arquivo'],
                    'seq': seq,
                    'action': action,
                    'info_adicional': info,
                    'diretorio_origem': row['diretorio_origem'],


                }
                linhas_df_analise.append(nova_linha)

    df_analise = pd.DataFrame(linhas_df_analise)
    return df_analise


In [83]:
df_analise = criar_df_analise(df_documento_recebido)

In [84]:
df_analise

,index,data_processamento,Batch,nome_arquivo,seq,action,info_adicional,diretorio_origem
0,1,06/09/2023 18:45:57,Batch_17,Nota Fiscal Eletrônica 1717 - BALL - BVI.PDF,1,ajustar_nome,nota_fiscal_eletronica_1717___ball___bvi.pdf,pipeline_extracao_documentos/2_documentos_para...
1,2,06/09/2023 18:45:57,Batch_17,NF 688 - ConsoR Zadar- Engetecnica (ENZA).pdf,1,ajustar_nome,nf_688___consor_zadar__engetecnica_enza.pdf,pipeline_extracao_documentos/2_documentos_para...
2,3,06/09/2023 18:45:57,Batch_17,1127 fogo em 20-07-23.pdf,1,ajustar_nome,1127_fogo_em_20_07_23.pdf,pipeline_extracao_documentos/2_documentos_para...
3,4,06/09/2023 18:45:57,Batch_17,NF 689- PMMacae- 8ª Med CO-22-22 TB Contrato 0...,1,ajustar_nome,nf_689__pmmacae__8a_med_co_22_22_tb_contrato_0...,pipeline_extracao_documentos/2_documentos_para...
4,5,06/09/2023 18:45:57,Batch_17,Nota Fiscal_229_Agosto-Schneider.pdf,1,ajustar_nome,nota_fiscal_229_agosto_schneider.pdf,pipeline_extracao_documentos/2_documentos_para...
5,6,06/09/2023 18:45:57,Batch_17,38 EDSON JOSE OLIVEIRA DOS SANTOS.pdf,1,ajustar_nome,38_edson_jose_oliveira_dos_santos.pdf,pipeline_extracao_documentos/2_documentos_para...
6,7,06/09/2023 18:45:57,Batch_17,12 - Ótica do Cizinho.pdf,1,ajustar_nome,12___otica_do_cizinho.pdf,pipeline_extracao_documentos/2_documentos_para...
7,8,06/09/2023 18:45:57,Batch_17,FLEXPRIN.pdf,1,ajustar_nome,flexprin.pdf,pipeline_extracao_documentos/2_documentos_para...
8,9,06/09/2023 18:45:57,Batch_17,NFS-e 35.pdf,1,ajustar_nome,nfs_e_35.pdf,pipeline_extracao_documentos/2_documentos_para...
9,9,06/09/2023 18:45:57,Batch_17,NFS-e 35.pdf,2,split_paginas,2,pipeline_extracao_documentos/2_documentos_para...


In [86]:
# DataFrame de análise
df_analise = pd.DataFrame(columns=['data_processamento', 'Batch', 'diretorio_origem', 'nome_arquivo', 'unique_ID', 'parent_ID', 'seq', 'action', 'nro_paginas'])

In [87]:
df_analise

,data_processamento,Batch,diretorio_origem,nome_arquivo,unique_ID,parent_ID,seq,action,nro_paginas


In [90]:
# Ao receber um novo documento
new_unique_id = generate_unique_id()

In [91]:
new_unique_id

'843c1013-9c84-4823-9a38-cc1ff7332d74'

In [92]:
# DataFrame de documentos recebidos
df_documento_recebido = pd.DataFrame(columns=['Batch', 'nome_arquivo', 'caminho_arquivo'])

In [93]:
# Preenche o df_documento_recebido com informações do novo documento
df_documento_recebido = df_documento_recebido.append({'Batch': 'Batch_17', 'nome_arquivo': 'Exemplo.pdf', 'caminho_arquivo': '/caminho/Exemplo.pdf'}, ignore_index=True)

/tmp/ipykernel_20823/2890610164.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_documento_recebido = df_documento_recebido.append({'Batch': 'Batch_17', 'nome_arquivo': 'Exemplo.pdf', 'caminho_arquivo': '/caminho/Exemplo.pdf'}, ignore_index=True)


In [94]:
# Preenche o df_analise com análises iniciais e o unique_ID
df_analise = df_analise.append({
    'data_processamento': pd.Timestamp.now(),
    'Batch': 'Batch_17',
    'diretorio_origem': '/caminho/',
    'nome_arquivo': 'Exemplo.pdf',
    'unique_ID': new_unique_id,
    'parent_ID': None,
    'seq': 1,
    'action': 'recebido',
    'nro_paginas': 2
}, ignore_index=True)


/tmp/ipykernel_20823/4173418973.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_analise = df_analise.append({


In [95]:
df_analise

,data_processamento,Batch,diretorio_origem,nome_arquivo,unique_ID,parent_ID,seq,action,nro_paginas
0,2023-09-06 19:18:19.157168,Batch_17,/caminho/,Exemplo.pdf,843c1013-9c84-4823-9a38-cc1ff7332d74,None,1,recebido,2


In [107]:
doc_path = "pipeline_extracao_documentos/2_documentos_para_extracao/21_aguardando_processamento/Batch_17/fwdnotasfaltantesnosistemadeemisso/Doria Marinho 0295 Carlos Leandro.pdf"

In [113]:
doc_path_2 = "pipeline_extracao_documentos/2_documentos_para_extracao/21_aguardando_processamento/Batch_17/fwdnotasfaltantesnosistemadeemisso/Doria Marinho 0295 Carlos Leandro copy.pdf"

In [116]:
msg_path = "pipeline_extracao_documentos/0_arquivos_teste_pipeline/emails/Notas Magé.msg"

In [109]:
hash_value = generate_file_hash(doc_path)

In [110]:
print(f"O hash do arquivo é: {hash_value}")

O hash do arquivo é: 53305daf18b4e42f36e27a8b900fae3a57cc033b402ba5d00c183782bc28b189


In [114]:
hash_value_2 = generate_file_hash(doc_path_2)

In [115]:
print(f"O hash do arquivo é: {hash_value_2}")

O hash do arquivo é: 2ec583e65c6cc2db60103ab3a4ebc7384414294bebfd081c6bb498ba9b7d17a5


In [117]:
hash_msg_value = generate_file_hash(msg_path)
print(f"O hash do arquivo é: {hash_msg_value}")

O hash do arquivo é: 3c9b796c1d5cb097d8aa95616ffd08864eebb55635313c4dfe59af9d73982a30


In [118]:
zip_path = "pipeline_extracao_documentos/1_emails_documentos_recebidos/12_attachments/fwdnotasfaltantesnosistemadeemisso.zip"

In [119]:
hash_zip_value = generate_file_hash(zip_path)
print(f"O hash do arquivo é: {hash_zip_value}")

O hash do arquivo é: ff1653e239202b20e56261e7f2b0fe91cb75009f92a83b2e3f2d4e08f1645ed9


In [ ]:
# Criar aqui um df_documento_recebido para teste
# df_documento_recebido = ...

# Chama a função e cria df_analise
# df_analise = criar_df_analise(df_documento_recebido)

In [62]:
# Iterar sobre cada linha do df_documento_recebido
for idx, row in df_documento_recebido.iterrows():
    # Iterar sobre cada ação da lista pre_processo
    file_name = row['nome_arquivo']
    print(f'\n\nidx: {idx} | file_name: {file_name}\n')
    for seq, action in enumerate(pre_processo, start=1):
        print(f'seq: {seq} | action: {action}')
        nova_linha = {
            'index': row['index'],
            'data_processamento': row['data_processamento'],
            'Batch': row['Batch'],
            'diretorio_origem': row['diretorio_origem'],
            'nome_arquivo': row['nome_arquivo'],
            'seq': seq,
            'action': action
        }
        
        linhas_df_analise.append(nova_linha)



idx: 0 | file_name: Nota Fiscal Eletrônica 1717 - BALL - BVI.PDF

seq: 1 | action: ajustar_nome
seq: 2 | action: split_paginas
seq: 3 | action: ajustar_imagem
seq: 4 | action: buscar_nome_prefeitura


idx: 1 | file_name: NF 688 - ConsoR Zadar- Engetecnica (ENZA).pdf

seq: 1 | action: ajustar_nome
seq: 2 | action: split_paginas
seq: 3 | action: ajustar_imagem
seq: 4 | action: buscar_nome_prefeitura


idx: 2 | file_name: 1127 fogo em 20-07-23.pdf

seq: 1 | action: ajustar_nome
seq: 2 | action: split_paginas
seq: 3 | action: ajustar_imagem
seq: 4 | action: buscar_nome_prefeitura


idx: 3 | file_name: NF 689- PMMacae- 8ª Med CO-22-22 TB Contrato 045-22.pdf

seq: 1 | action: ajustar_nome
seq: 2 | action: split_paginas
seq: 3 | action: ajustar_imagem
seq: 4 | action: buscar_nome_prefeitura


idx: 4 | file_name: Nota Fiscal_229_Agosto-Schneider.pdf

seq: 1 | action: ajustar_nome
seq: 2 | action: split_paginas
seq: 3 | action: ajustar_imagem
seq: 4 | action: buscar_nome_prefeitura


idx: 5 |

In [ ]:
# Criar o novo DataFrame
df_analise = pd.DataFrame(linhas_df_analise)

In [ ]:
# def criar_df_analise(df_documento_recebido):
#     linhas_df_analise = []
    
#     for idx, row in df_documento_recebido.iterrows():
#         file_name = row['nome_arquivo']
#         file_path = row['diretorio_origem']

#         # Chama a função analisa_nro_pages
#         doc_1_page, nro_paginas = analisa_nro_pages(file_path)

#         # Chama a função conv_filename
#         new_name = conv_filename(file_name)

#         acoes_necessarias = []

#         # Se o documento tem mais de uma página
#         if not doc_1_page:
#             acoes_necessarias.append(('split_paginas', nro_paginas))

#         # Se o nome do arquivo precisa ser ajustado
#         if new_name != file_name:
#             acoes_necessarias.append(('ajustar_nome', new_name))

#         for seq, (action, info) in enumerate(acoes_necessarias, start=1):
#             nova_linha = {
#                 'index': row['index'],
#                 'data_processamento': row['data_processamento'],
#                 'Batch': row['Batch'],
#                 'nome_arquivo': row['nome_arquivo'],
#                 'seq': seq,
#                 'action': action,
#                 'info_adicional': info,  # Para armazenar informações como nro_paginas ou new_name
#                 'diretorio_origem': row['diretorio_origem']

#             }
#             linhas_df_analise.append(nova_linha)

#     df_analise = pd.DataFrame(linhas_df_analise)
#     return df_analise




In [ ]:
pre_processo = ['ajustar_nome', 'split_paginas', 'ajustar_imagem', 'buscar_nome_prefeitura']

# Lista vazia para armazenar as linhas do novo DataFrame
linhas_df_analise = []


In [ ]:
def analisar_pdf(args):
    # Lógica para analisar o PDF e retornar uma lista de ações necessárias
    acoes = []
    if args['numero_paginas'] > 1:
        acoes.append('split_paginas')
    if args['nome_precisa_ajuste']:
        acoes.append('ajustar_nome')
    # Adicionar outras verificações aqui
    return acoes


In [ ]:

linhas_df_analise = []

for idx, row in df_documento_recebido.iterrows():
    # Realizar a análise inicial para determinar quais ações são necessárias
    acoes_necessarias = analisar_pdf(row)
    
    for seq, action in enumerate(acoes_necessarias, start=1):
        nova_linha = {
            'index': row['index'],
            'data_processamento': row['data_processamento'],
            'Batch': row['Batch'],
            'diretorio_origem': row['diretorio_origem'],
            'nome_arquivo': row['nome_arquivo'],
            'seq': seq,
            'action': action
        }
        linhas_df_analise.append(nova_linha)
    


In [ ]:
# 2. ZZZ Estou analisando a quantidade de paginas 
one_page_doc, paginas = analisa_nro_pages(new_path_name)

# 2. ZZZ Faço um If para tomar de decisao do documentos
if not one_page_doc:
    print(f'documento com varias paginas = {paginas} ser splitado')
else:    
    print(f'documento apenas {paginas} pagina e nao preciosa ser splitado')

In [ ]:
# 5. XXX Estabelece o pipeline de documentos
def ajusta_pipeline(index, batch_name, file, documentos_extracao_path, folder_name, file_path):
    
    time_now = cron.timenow_pt_BR()
    # 1. Processo de ajuste de nome do documento
    new_name = conv_filename(file)
    #print(f'file: {file} | new_name: {new_name}')
    root_dir_batch = os.path.join(documentos_extracao_path, batch_name)
    root_dir_batch_dir = os.path.join(root_dir_batch, folder_name)
    new_name_file_path = os.path.join(root_dir_batch_dir, new_name)
    if new_name != file:
        #print(f'nome mudou | {folder_name} | file: {file} | new_name: {new_name}\n{file_path}\n{new_name_file_path}\n')
        status_pro = 'NOME AJUSTADO'
        try:
            shutil.move(file_path, new_name_file_path)
            status_pro = 'NOME AJUSTADO'
            new_path_name = new_name_file_path
        except Exception as e:
            print(f"Erro ao mudar nome do arquivo: {e}")  
    else:
        status_pro = 'processado'
        #print(f'Nao mudou | {folder_name}  | file: {file} | new_name: {new_name}')
        
        new_path_name = file_path
    
    # 2. Verifica nro de paginas
    one_page_doc, paginas = analisa_nro_pages(new_path_name)
    # 2. ZZZ Faço um If para tomar de decisao do documentos
    if not one_page_doc:
        try:
            print(f'documento com varias paginas = {paginas} ser splitado')
            # 3. Processa o split de paginas
            new_pdf_path_file, nome_file_deletar = split_pdf_pages(new_path_name)
            
            # 3.1 elimina documento com varias paginas
            if new_pdf_path_file:
                os.remove(nome_file_deletar)
            
            df_split = processa_pipeline()   
        except Exception as e:
            print(f"Erro ao efetuar split de paginas: {e}")       
    else:    
        print(f'documento apenas {paginas} pagina e nao preciosa ser splitado')

    return {
        'index': index,
        'data_processamento': time_now,
        'Batch': batch_name,
        'diretorio_origem': folder_name,
        'nome_arquivo_origem': file,
        'nome_arquivo_destino': new_name,
        'one_page_doc': one_page_doc,
        'numero_paginas': paginas,
        'status': status_pro,
    }


# 3. FunÇao para ajustar documentos no pipeline antes de serem processados
def processa_pipeline():
    rows_list = []
    i = 1
    for root, dirs, files in os.walk(documentos_extracao_path):
        folder_name = os.path.basename(root)
        for file in files:
            file_path = os.path.join(root, file)
            try:
                new_row = ajusta_pipeline(i, batch_name, file, documentos_extracao_path, folder_name, file_path)
                rows_list.append(new_row)
            except Exception as e:
                print(f"Erro ao criar nova row: {e}") 
        i += 1
    
    df_processa_pipe = pd.DataFrame(rows_list)
    
    return df_processa_pipe